In [8]:
#! pip install yahoo_fin

import requests
import re
import pandas as pd
import numpy as np
from yahoo_fin.stock_info import get_data, tickers_sp500, tickers_nasdaq, tickers_other, get_quote_table
import bs4 as bs
import datetime as dt
import functools


def create_indicator(url):
    # url = 'https://www.alphavantage.co/query?function=DEMA&symbol=TSLA&interval=daily&time_period=10&series_type=open&apikey=E77UTWHMIKTD3P60'
    r = requests.get(url)
    data = r.json()
    # print(data)
    
    
    df = pd.DataFrame(data = data)
    # print(df)     # create dataframe from json dictionary
    df['Symbol'] = df['Meta Data'][0]  # create new column consisting the Symbol 
    df.drop('Meta Data', inplace = True, axis = 1)
    df.dropna(inplace =True)
    
    # df.reset_index(inplace = True)
    # df.rename(columns = {'index': 'ngay'}, inplace =True)
    tech = df.iloc[:, 0].name   # tech is indicator name
    tech_1 = tech[20:]
    df.rename(columns = {tech: tech_1}, inplace = True)  # renane to tidy name
    df.reset_index(inplace = True)
    df.rename(columns = {'index': 'ngay'}, inplace =True)
    
    lit = []
    for i in range(len(df)):
        a = str(df[tech_1][i])
        x = re.findall("[0-9]", a)
        lit.append(''.join(x))
    df[tech_1] = lit
    
    return df

In [3]:
# # function to get 500 name tickers
# def save_sp500_tickers():
#     resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
#     soup = bs.BeautifulSoup(resp.text, 'lxml')
#     table = soup.find('table', {'class': 'wikitable sortable'})
#     tickers = []
#     for row in table.findAll('tr')[1:]:
#         ticker = row.findAll('td')[0].text
#         tickers.append(ticker)
#     tickers_new = []
#     for i in range(len(tickers)):
#         tickers[i] = tickers[i][0:-1]
#         tickers_new.append(tickers[i])
    
#     return tickers_new
# tickers = save_sp500_tickers()

Symbol = 'GOOGL'

In [9]:
# def merge_indi(Symbol):
    
url_1 = 'https://www.alphavantage.co/query?function='
url_3 = '&symbol='
url_5 = '&interval=daily&time_period=10&series_type=open&apikey=E77UTWHMIKTD3P60'

lit_indi_1 =['RSI', 'SMA', 'EMA','WMA' , 'MOM']
# lit_indi_2 = ['ROC','CMO','AROON','AD','OBV']
# lit_indi_2  = ['SMA','EMA' ,'APO','CCI']
# lit_indi_4 = ['MOM','PPO','ADX']

# create list of list indi
# lit_indi = [lit_indi_1 , lit_indi_2 ]

# Create the merge dataframe 1
lit_1 = []
for i in range(len(lit_indi_1)):
    url = url_1 + lit_indi_1[i] + url_3 + Symbol + url_5
    lit_1.append(url)
    
lit_2 = list()
for j in range(len(lit_1)):
    df = create_indicator(lit_1[j])
    lit_2.append(df)

df_merge_total = functools.reduce(lambda left , right: pd.merge(left, right,
                                    on= ['ngay','Symbol']),lit_2)
print(df_merge_total)
      


# lay gia mo cua , dong cua, high , low tu yahoo_finance

# df_yahoo = si.get_data('GOOGL')   # df is a dataframe
# df_yahoo.reset_index(inplace = True) 
# df_yahoo.rename(columns = {'ticker': 'Symbol', 'index': 'ngay'}, inplace = True)


# df_merge_lit = [df_merge_1, df_merge_2  ]

# df_merge_total = functools.reduce(lambda left , right: pd.merge(left, right,
#                                 on= ['ngay','Symbol']),df_merge_lit)

# print(df_merge_total) # la df nhung chi bao


            ngay     RSI Symbol       SMA       EMA       WMA      MOM
0     2022-02-09  527613  GOOGL  27947193  28042946  28338018  1968900
1     2022-02-08  499034  GOOGL  27750303  28009667  28257582  2057300
2     2022-02-07  584921  GOOGL  27544573  28055082  28210177  3654400
3     2022-02-04  574765  GOOGL  27179133  27878434  27906383  2170227
4     2022-02-03  612635  GOOGL  26962110  27698324  27592416  1891100
...          ...     ...    ...       ...       ...       ...      ...
4386  2004-09-09  549049  GOOGL    515411    514502    510731    11335
4387  2004-09-08  512756  GOOGL    516545    514594    511178    52663
4388  2004-09-07  518518  GOOGL    521811    516668    514187    48901
4389  2004-09-03  517434  GOOGL    526701    518902    517839    00301
4390  2004-09-02  485075  GOOGL    526731    521700    521552    04614

[4391 rows x 7 columns]


In [10]:
df_yahoo = get_data(Symbol)   # df is a dataframe
# df_yahoo.reset_index(inplace = True) 
df_yahoo = df_yahoo.iloc[::-1]  # doi nguoc dataframe
# vi du lieu yahoo_fin ngan hon nen phai cat df
df_yahoo = df_yahoo[:len(df_merge_total)]

df_yahoo.reset_index(inplace =True)
df_yahoo.rename(columns = {'ticker': 'Symbol', 'index': 'ngay'}, inplace = True)  # doi ten

df_yahoo.reset_index(inplace = True)
df_yahoo.drop(columns = ['index' ] , inplace = True)
df_yahoo

,ngay,open,high,low,close,adjclose,volume,Symbol
0,2022-02-09,2819.270020,2850.489990,2803.739990,2831.840088,2831.840088,2005200,GOOGL
1,2022-02-08,2780.530029,2800.000000,2740.280029,2787.979980,2787.979980,2714200,GOOGL
2,2022-02-07,2885.000000,2885.000000,2778.000000,2784.020020,2784.020020,2748700,GOOGL
3,2022-02-04,2868.889893,2897.500000,2810.300049,2865.860107,2865.860107,2897200,GOOGL
4,2022-02-03,2914.110107,2993.500000,2853.310059,2861.800049,2861.800049,3575900,GOOGL
...,...,...,...,...,...,...,...,...
4386,2004-09-09,51.316315,51.406406,50.550552,51.206207,51.206207,4061734,GOOGL
4387,2004-09-08,50.420422,51.566566,50.300301,51.201202,51.201202,4985609,GOOGL
4388,2004-09-07,50.555557,51.051052,49.854855,50.840839,50.840839,5847547,GOOGL
4389,2004-09-03,50.525524,50.920921,49.709709,50.055054,50.055054,5152442,GOOGL


In [11]:
df_merge_total

,ngay,RSI,Symbol,SMA,EMA,WMA,MOM
0,2022-02-09,527613,GOOGL,27947193,28042946,28338018,1968900
1,2022-02-08,499034,GOOGL,27750303,28009667,28257582,2057300
2,2022-02-07,584921,GOOGL,27544573,28055082,28210177,3654400
3,2022-02-04,574765,GOOGL,27179133,27878434,27906383,2170227
4,2022-02-03,612635,GOOGL,26962110,27698324,27592416,1891100
...,...,...,...,...,...,...,...
4386,2004-09-09,549049,GOOGL,515411,514502,510731,11335
4387,2004-09-08,512756,GOOGL,516545,514594,511178,52663
4388,2004-09-07,518518,GOOGL,521811,516668,514187,48901
4389,2004-09-03,517434,GOOGL,526701,518902,517839,00301


In [13]:
df_final_GOOGL = pd.concat([df_yahoo, df_merge_total] , axis = 1)
df_final_GOOGL.columns

Index(['ngay', 'open', 'high', 'low', 'close', 'adjclose', 'volume', 'Symbol',
       'ngay', 'RSI', 'Symbol', 'SMA', 'EMA', 'WMA', 'MOM'],
      dtype='object')

In [14]:
# Data cuoi cua GOOGL
df_final_GOOGL.drop(df_final_GOOGL.columns[7],axis =1, inplace =True)
# drop 'ngay' column by name
# df_final_GOOGL.drop(df_final_GOOGL.columns[0], axis = 1, inplace =True)
df_final_GOOGL


,ngay,open,high,low,close,adjclose,volume,ngay,RSI,SMA,EMA,WMA,MOM
0,2022-02-09,2819.270020,2850.489990,2803.739990,2831.840088,2831.840088,2005200,2022-02-09,527613,27947193,28042946,28338018,1968900
1,2022-02-08,2780.530029,2800.000000,2740.280029,2787.979980,2787.979980,2714200,2022-02-08,499034,27750303,28009667,28257582,2057300
2,2022-02-07,2885.000000,2885.000000,2778.000000,2784.020020,2784.020020,2748700,2022-02-07,584921,27544573,28055082,28210177,3654400
3,2022-02-04,2868.889893,2897.500000,2810.300049,2865.860107,2865.860107,2897200,2022-02-04,574765,27179133,27878434,27906383,2170227
4,2022-02-03,2914.110107,2993.500000,2853.310059,2861.800049,2861.800049,3575900,2022-02-03,612635,26962110,27698324,27592416,1891100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4386,2004-09-09,51.316315,51.406406,50.550552,51.206207,51.206207,4061734,2004-09-09,549049,515411,514502,510731,11335
4387,2004-09-08,50.420422,51.566566,50.300301,51.201202,51.201202,4985609,2004-09-08,512756,516545,514594,511178,52663
4388,2004-09-07,50.555557,51.051052,49.854855,50.840839,50.840839,5847547,2004-09-07,518518,521811,516668,514187,48901
4389,2004-09-03,50.525524,50.920921,49.709709,50.055054,50.055054,5152442,2004-09-03,517434,526701,518902,517839,00301


In [15]:
df_final_GOOGL.to_csv('GOOGL.csv')

In [16]:
df_final_GOOGL

,ngay,open,high,low,close,adjclose,volume,ngay,RSI,SMA,EMA,WMA,MOM
0,2022-02-09,2819.270020,2850.489990,2803.739990,2831.840088,2831.840088,2005200,2022-02-09,527613,27947193,28042946,28338018,1968900
1,2022-02-08,2780.530029,2800.000000,2740.280029,2787.979980,2787.979980,2714200,2022-02-08,499034,27750303,28009667,28257582,2057300
2,2022-02-07,2885.000000,2885.000000,2778.000000,2784.020020,2784.020020,2748700,2022-02-07,584921,27544573,28055082,28210177,3654400
3,2022-02-04,2868.889893,2897.500000,2810.300049,2865.860107,2865.860107,2897200,2022-02-04,574765,27179133,27878434,27906383,2170227
4,2022-02-03,2914.110107,2993.500000,2853.310059,2861.800049,2861.800049,3575900,2022-02-03,612635,26962110,27698324,27592416,1891100
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4386,2004-09-09,51.316315,51.406406,50.550552,51.206207,51.206207,4061734,2004-09-09,549049,515411,514502,510731,11335
4387,2004-09-08,50.420422,51.566566,50.300301,51.201202,51.201202,4985609,2004-09-08,512756,516545,514594,511178,52663
4388,2004-09-07,50.555557,51.051052,49.854855,50.840839,50.840839,5847547,2004-09-07,518518,521811,516668,514187,48901
4389,2004-09-03,50.525524,50.920921,49.709709,50.055054,50.055054,5152442,2004-09-03,517434,526701,518902,517839,00301
